<H3>PRI 2022: first project delivery</H3>

**GROUP X**
- name, number
- name, number
- name, number

<H3>Part I: demo of facilities</H3>

In [71]:
from IPython.core.display import HTML

#imports

from mainFunctions.ranking import ranking
from operator import attrgetter
from helper.documentHelper import read_files
from mainFunctions.indexing import indexing
from src.helper.mockDataVisualize import transfer_function_output_ranking
from src.mainFunctions.visualize import run_visualize

**A) Indexing facilities**

*A.1 Preprocessing options: statistics*

In [72]:
#rare, most common terms


*A.2 Indexing statistics*

In [73]:
#code, statistics and/or charts here
#time, memory

**B) Ranking facilities**

In [74]:
# load files

order_ranked = True
text_processing = True
max_sent = 8
max_chars = 1010
documents = read_files(text_processing, ["business"])

corpus_index = indexing(list(map(attrgetter('text_terms'), documents)))
corpus_idfs: {str: float} = {}

for v in corpus_index:
    corpus_idfs[v] = corpus_index[v].inverted_document_frequency

document = documents[0]

summary1 = ranking(document, max_sent, max_chars, order_ranked, corpus_idfs, {"rank_option": "tf", "mmr": False})
summary2 = ranking(document, max_sent, max_chars, order_ranked, corpus_idfs, {"rank_option": "tf-idf", "mmr": False})
summary3 = ranking(document, max_sent, max_chars, order_ranked, corpus_idfs, {"rank_option": "bm25", "mmr": False})
summary4 = ranking(document, max_sent, max_chars, order_ranked, corpus_idfs, {"rank_option": "rrf", "mmr": False})
summary5 = ranking(document, max_sent, max_chars, order_ranked, corpus_idfs, {"rank_option": "tf-idf", "mmr": True})

*B.1 Summarization solution: results for a given document*

In [75]:
precision, recall = calculate_precision_recall()

*B.2 Classic IR models: differences*

In [76]:
#code, statistics and/or charts here

*B.3 Reciprocal rank funsion*

In [77]:
#code, statistics and/or charts here

*B.4 Maximal Marginal Relevance*

In [78]:
#code, statistics and/or charts here

**C) Sentence higlighting facilities**

In [85]:
dict = transfer_function_output_ranking(document.text_sentences, summary1)
doc_path = "summaries/summary1"
run_visualize(doc_path, dict, 3)
HTML(filename=doc_path + ".html")

["Users of Apple's music jukebox iTunes need to update the software to avoid a potential security threat.", 'Hackers can build malicious playlist files which could crash the program and let them seize control of the computer by inserting Trojan code.', 'A new version of iTunes is now available from the Apple website which solves the problem.', 'Security firm iDefence, which notified users of the problem, recommended that users upgrade to iTunes version 4.7.1.', 'The problem affects all users of iTunes - Windows and Mac OS - running versions 4.7 and earlier.', 'Users can automatically upgrade iTunes by opening the "look for updates" window in the program.', 'The security firm says users should avoid clicking on or accessing playlist files - which have the file extension of .pls or .m3u - which have come from unknown sources.', "Itunes is the world's most popular online music store with more than 200 million songs downloaded since it launched in 2003."]


['"The pressure on costs and mar

**D) Evaluation facilities**

*D.1 Evaluation options*

In [80]:
#code, statistics and/or charts here

*D.2 Comparison of settings (IR models, preprocessing)*

In [81]:
#code, statistics and/or charts here

<H3>Part II: questions materials (optional)</H3>

**(a)** Corpus and summary description. Distribution of informative terms before and after text processing.

In [82]:
#code, statistics and/or charts here

from src.helper.documentHelper import read_files


documents_no_preprocessing = read_files(False)
documents_preprocessing = read_files(True)

print(documents_preprocessing[0].text)  #Test

# Calculate distribution of informative termsstics and/or charts here

The UK manufacturing sector will continue to face "serious challenges" over the next two years, the British Chamber of Commerce (BCC) has said.

The group's quarterly survey of companies found exports had picked up in the last three months of 2004 to their best levels in eight years. The rise came despite exchange rates being cited as a major concern. However, the BCC found the whole UK economy still faced "major risks" and warned that growth is set to slow. It recently forecast economic growth will slow from more than 3% in 2004 to a little below 2.5% in both 2005 and 2006.

Manufacturers' domestic sales growth fell back slightly in the quarter, the survey of 5,196 firms found. Employment in manufacturing also fell and job expectations were at their lowest level for a year.

"Despite some positive news for the export sector, there are worrying signs for manufacturing," the BCC said. "These results reinforce our concern over the sector's persistent inability to sustain recovery." The o

**(b)** Summarization performance for the overall and category-conditional corpora.

In [83]:
#code, statistics and/or charts here

**...** (additional questions with empirical results)

<H3>Part III: Other caluclations to support the project</H3>

In [84]:
# get average sentences and characters of the reference summaries
from helper.documentHelper import read_files
totalSentences = 0
totalCharacters = 0
totalDocuments = 0
documents = read_files(False)

for document in documents:
    sentences = document.referenceSummary
    totalSentences += len(sentences)
    totalCharacters += len(" ".join(sentences))
    totalDocuments +=1

print("avg sentences", totalSentences / totalDocuments)
print("avg characters", totalCharacters /totalDocuments)

avg sentences 7.893483146067416
avg characters 1008.9977528089887
avg sentences 7.893483146067416
avg characters 1008.9977528089887


<H3>END</H3>